In [2]:
pip install pandas_ta

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 115.1/115.1 kB 3.4 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Created wheel for pandas_ta: filename=pandas_ta-0.3.14b0-py3-none-any.whl size=218909 sha256=46b9499bb4b983cbea5e1b7afac4f704bca106ba391f460e3b59019b7d6935a6
  Stored in directory: /root/.cache/pip/wheels/69/00/ac/f7fa862c34b0e2ef320175100c233377b4c558944f12474cf0
Successfully built pandas_ta


In [4]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import pandas_ta as ta

# Load data
data = pd.read_csv('/content/ETHUSDT_2h.csv')
data['datetime'] = data['datetime'].astype('datetime64[s]')
data = data.set_index('datetime')

# RSI Calculation
def RSI(data, window=14):
    delta = data.diff(1)
    gain = (delta.where(delta > 0, 0)).rolling(window=window).mean()
    loss = (-delta.where(delta < 0, 0)).rolling(window=window).mean()
    rs = gain / loss
    return 100 - (100 / (1 + rs))

data['RSI'] = RSI(data['close'], window=14)

# EMA Calculations
data['9_EMA'] = data['close'].ewm(span=9, adjust=False).mean()
data['16_EMA'] = data['close'].ewm(span=16, adjust=False).mean()
data['20_EMA'] = data['close'].ewm(span=20, adjust=False).mean()
data['65_EMA'] = data['close'].ewm(span=65, adjust=False).mean()
data['200_EMA'] = data['close'].ewm(span=100, adjust=False).mean()
data['40_EMA'] = data['close'].ewm(span=60, adjust=False).mean()
data['60_SMA'] = data['close'].rolling(window=60).mean()

# Hawkes Process Function
def hawkes_process(data: pd.Series, kappa: float):
    assert kappa > 0.0
    alpha = np.exp(-kappa)
    arr = data.to_numpy()
    output = np.zeros(len(data))
    output[:] = np.nan
    for i in range(1, len(data)):
        output[i] = output[i-1] * alpha + arr[i] if not np.isnan(output[i-1]) else arr[i]
    return pd.Series(output, index=data.index) * kappa

# Volume Signal Function
def vol_signal(close: pd.Series, vol_hawkes: pd.Series, lookback: int):
    signal = np.zeros(len(close))
    q95 = vol_hawkes.rolling(lookback).quantile(0.95)
    curr_sig = 0
    a = 0

    for i in range(len(signal)):
        if a < close.iloc[i]:
            a = close.iloc[i]

        if vol_hawkes.iloc[i - 1] <= q95.iloc[i - 1] and vol_hawkes.iloc[i] > q95.iloc[i] and data['RSI'].iloc[i] > 55:
            if curr_sig == 0:
                signal[i] = 1
                curr_sig = 1

        elif curr_sig == 1 and (data['RSI'].iloc[i] > 95 or data['RSI'].iloc[i] < 30 or data['close'].iloc[i] < data['60_SMA'].iloc[i]):
            signal[i] = -1
            curr_sig = 0

    return signal

# Normalize volume and calculate Hawkes process
norm_lookback = 14
data['atr'] = ta.atr(np.log(data['high']), np.log(data['low']), np.log(data['close']), norm_lookback)
data['norm_range'] = (np.log(data['high']) - np.log(data['low'])) / data['atr']

data['v_hawk'] = hawkes_process(data['norm_range'], 0.002)
data['signals'] = vol_signal(data['close'], data['v_hawk'], 55)

# Filtered data with signals
filtered_data = data[data['signals'] != 0][['close', 'high', 'low', 'open', 'signals']]

# Calculate returns based on entries and exits
filtered_data1 = filtered_data.copy()
filtered_data1['return1'] = 0.0
entry_price = None

for i in range(1, len(filtered_data1)):
    if filtered_data1['signals'].iloc[i] == 1:  # Enter trade
        entry_price = filtered_data1['close'].iloc[i]
    elif filtered_data1['signals'].iloc[i] == -1 and entry_price is not None:  # Exit trade
        exit_price = filtered_data1['close'].iloc[i]
        trade_return = (exit_price - entry_price) / entry_price
        filtered_data1['return1'].iloc[i] = trade_return
        entry_price = None

# Calculate compounded return
filtered_data1['compounded_return'] = (1 + filtered_data1['return1']).cumprod() - 1

# Save filtered data to CSV (no changes to column names)
filtered_data.to_csv("Strategy2ETH2467_Profit.csv")


<ipython-input-4-1ecb23a0288e>:85: FutureWarning: ChainedAssignmentError: behaviour will change in pandas 3.0!
You are setting values through chained assignment. Currently this works in certain cases, but when using Copy-on-Write (which will become the default behaviour in pandas 3.0) this will never work to update the original DataFrame or Series, because the intermediate object on which we are setting values will behave as a copy.
A typical example is when you are setting values in a column of a DataFrame, like:

df["col"][row_indexer] = value

Use `df.loc[row_indexer, "col"] = values` instead, to perform the assignment in a single step and ensure this keeps updating the original `df`.

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy

  filtered_data1['return1'].iloc[i] = trade_return
<ipython-input-4-1ecb23a0288e>:85: FutureWarning: ChainedAssignmentError: behaviour will change in pandas 3.0!
